In [111]:
import collections.abc
collections.Mapping = collections.abc.Mapping
collections.MutableMapping = collections.abc.MutableMapping
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import numpy as np
import tensorflow as tf
import json
import random
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="Model was constructed with shape (None, None, 47, 121) for input KerasTensor.*")





In [112]:

    
def check_sentence(sentence,vocab):
    word_freq = Counter(vocab)
    prob = {}
    total = sum(word_freq.values())
    for k in word_freq:
        prob[k] = word_freq[k]/total 
    question = []
    for word in sentence:
        word = word.lower()
        if word in word:
            question.append(word)
        else:
            sim = [1-(textdistance.Jaccard(qval=2).distance(v,word)) for v in word_freq.keys()]
            df = pd.DataFrame.from_dict(prob, orient='index').reset_index()
            df = df.rename(columns={'index':'Word', 0:'Prob'})
            df['Similarity'] = sim
            output = df.sort_values(['Similarity', 'Prob'], ascending=False).head()
            return(output)   

def tokenize(question):
    return word_tokenize(question)
    
def stem_word(word, stemmer = PorterStemmer()):
    return stemmer.stem(word)

def data():
    with open("tags.json", "r") as file:
        tags = json.load(file)
    with open("all_words.json", "r") as file:
        all_words = json.load(file)
    return tags,all_words


def bag_of_words(vocab,tok_sentence):
    bag = np.zeros(len(vocab), dtype=np.float32)
    for ind, word in enumerate(vocab):
        if word in tok_sentence:
            bag[ind] = 1.0
    return bag

def predict(input, tags):
    model = tf.keras.models.load_model("model1")
    prediction = tags[np.argmax(model.predict(input))]
    return prediction
      

def correct_sentence(question):
    pass

def question_understood(question):
    pass

def get_answer(prediction):
    with open("intents.json", "r") as data:
        intents = json.load(data)
        for tg in intents["intents"]:
            if tg['tag'] == prediction:
                choice = random.choice(tg["responses"])
                break
    return choice


def process_question(question, vocab):
    tok_ques = tokenize(question)
    stemmed_words = [stem_word(s) for s in tok_ques if s not in [',', '.', '!', '?', ',']]
    word_bag = np.array([1.0 if word in stemmed_words else 0.0 for word in vocab], dtype=np.float32)
    word_bag = np.expand_dims(word_bag, axis=0)
    return word_bag





In [113]:
tags, vocab = data()
chat = True
print("Welcome to arc solution chatbot!")
while(chat):
    question = input(" ")
    if (question == "end"):
        print("Thanks for using this bot!")
        chat = False
        break
    print("User:\t", question)
    answer = get_answer(predict(process_question(question,vocab),tags))
    print("Answer:\t", answer)


Welcome to arc solution chatbot!
User:	 hi
1/1 [==============================] - 0s 201ms/step
Answer:	 Customer satisfaction is our top priority, and we strive to exceed our customers' expectations in every project we undertake.
User:	 good day
1/1 [==============================] - 0s 187ms/step
Answer:	 Hi! How may I help you?
User:	 i need to know your services
1/1 [==============================] - 0s 229ms/step
Answer:	 I'm sorry for the issues with the recruitment process. Let's discuss your concerns and find a resolution.
User:	 .
1/1 [==============================] - 0s 230ms/step
Answer:	 Arc Solution is an emblem of quality due to our unwavering commitment to excellence and customer satisfaction.
Thanks for using this bot!


In [114]:
check_sentence

<function __main__.check_sentence(sentence, vocab)>